In [1]:
print('Analysis Dashboard')

Analysis Dashboard


In [2]:
from ipywidgets import interact, interactive, widgets, interact_manual
import glob
import pandas as pd
import plotly.graph_objects as go

from scipy.signal import savgol_filter
import peakutils

from ipyfilechooser import FileChooser
%matplotlib widget

In [3]:
def plot(thresh, peakdist, buffer, polynomial, window):
    peaks = []
    basepoints = []
    #data = abs(vid.origin)
    print(len(vid.origin))
    
    for i in range(0, len(vid.origin)):
        vid.dataframe[i]['disp'] = savgol_filter(vid.origin[i]['disp'], window, polynomial)
        vid.dataframe[i]['disp'] = vid.dataframe[i]['disp'] * -1
        peaks.append(peakutils.indexes(vid.dataframe[i]['disp'], thresh, peakdist))
        peaks[i] = peaks[i][1:-1]
        basepoints.append([])
        for peak in peaks[i]:
            for k in range(peak - buffer, 1, -1):
                dfdt = (vid.dataframe[i]['disp'][k] - vid.dataframe[i]['disp'][k - 1])
                if dfdt<=0:
                    basepoints[i].append(k)
                    break
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=vid.dataframe[i]['time'],y=vid.dataframe[i]['disp'])) 
        fig.add_trace(go.Scatter(x=vid.dataframe[i]['time'][peaks[0]], y=vid.dataframe[i]['disp'][peaks[0]], mode = 'markers'))
        fig.add_trace(go.Scatter(x=vid.dataframe[i]['time'][basepoints[0]], y=vid.dataframe[i]['disp'][basepoints[0]], mode = 'markers'))
        fig.show()


In [4]:
class fileselection():
    def __init__(self):
        self.fc = FileChooser('/srv/scratch/Data/tracking_generated/')
        display(self.fc)

        self.fc.use_dir_icons = True
        self.fc.title = '<b>Choose Folder</b>'
        self.fc.register_callback(self.folderselected)
        
    def folderselected(self, chooser):
        display('asasa')
        self.dataframe = []
        self.origin = []
        files = glob.glob(self.fc.selected_path + '/*')
        print('aaaaaaaaa')
        for i in range(len(files)):
            self.dataframe.append([])
            self.origin.append([])
            self.dataframe[i] =  pd.read_csv(files[i])
            self.origin[i] = pd.read_csv(files[i])
            print(i)
        print(len(vid.origin))
        #self.paramaterization(.6, 10, 5, 3, 13)
vid = fileselection()


FileChooser(path='/srv/scratch/Data/tracking_generated', filename='', show_hidden='False')

In [5]:
@interact(thres = (0, 1, .1), peakdist = (0, 30), buffer = (0, 20), poly = (1, 16), window = (9, 31, 2))
def paramaterization(thres = .6, peakdist = 10, buffer = 5, poly = 3, window = 13):
    if vid.fc.selected_path is not None:
        plot(thres, peakdist, buffer, poly, window)
        

interactive(children=(FloatSlider(value=0.6, description='thres', max=1.0), IntSlider(value=10, description='p…

In [6]:

    #print(len(vid.origin))

In [7]:
%%html
<a href="https://costa.jack.engineering/user/jack/tree/scratch" target=_blank>Files</a>

In [7]:
#class fileselection():
#    def __init__(self):
#        self.folders = glob.glob('Data/tracking_generated/*')
#        self.filepath = None
#        
#        for i in range(len(self.folders)):
#            self.folders[i] = self.folders[i].split('/')[2]

#        display(interactive(self.loadfiles, Folders=self.folders))

#    def choosevideo(self, Data):
#        global dataframe, origin
#        self.filepath = Data
#        self.dataframe = self.origin = pd.read_csv(Data)
#        #print(self.origin)
#
#
#    def loadfiles(self, Folders):
#        files = sorted(glob.glob('Data/tracking_generated/' + Folders + '/*.csv'))
#        display(interactive(self.choosevideo, Data=files))
#vid = fileselection()

In [ ]:
#def folderselected(chooser):
#    display('asasa')
##    dataframe = []
 #   origin = []
 #   files = glob.glob(chooser.selected_path + '/*')
 #   print('aaaaaaaaa')
 #   for i in range(len(files)):
 #       dataframe.append([])
 ##       origin.append([])
  #      dataframe[i] =  pd.read_csv(files[i])
  #      origin[i] = pd.read_csv(files[i])
  #      print(i)
        
#@interact(thres = (0, 1, .1), peakdist = (0, 30), buffer = (0, 20), poly = (1, 16), window = (9, 31, 2))
#def someething(thres = .6, peakdist = 10, buffer = 5, poly = 3, window = 13):
#    fc = FileChooser('/srv/scratch/Data/tracking_generated/')
#    display(fc)#
#

  #  fc.use_dir_icons = True
  #  fc.title = '<b>Choose Folder</b>'
  #  fc.register_callback(folderselected) 
  #  if fc.selected_path is not None:
  #          plot(thres, peakdist, buffer, poly, window)
